In [1]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

# import pororo
# from pororo import Pororo


import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
There is no [win32] chromedriver for browser 100.0.4896 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Admin\.wdm\drivers\chromedriver\win32\100.0.4896.60]
C:\Users\Admin\AppData\Local\Temp\ipykernel_7312\3969098803.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)


In [3]:
# Crawling
def kor_to_trans(df, trans_lang, start_index,final_index):
    encode_sentence = []
    encode_subject = []
    encode_object = []
    
    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

    for i in tqdm(range(start_index,final_index)): 
    
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+df["sentence"].iloc[i])
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.8)
            backtrans_sentence = element.text 

            if (backtrans_sentence=='')|(backtrans_sentence==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans_sentence = element.text 
                encode_sentence.append(backtrans_sentence)
            else:
                encode_sentence.append(backtrans_sentence)
            
        except:
            encode_sentence.append('')
            
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+extract_word(df['subject_entity'].iloc[i]))
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.8)
            backtrans_subject = element.text 

            if (backtrans_subject=='')|(backtrans_subject==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans_subject = element.text 
                encode_subject.append(backtrans_subject)
            else:
                encode_subject.append(backtrans_subject)
            
        except:
            encode_subject.append('')
            
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+extract_word(df['object_entity'].iloc[i]))
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.8)
            backtrans_object = element.text 

            if (backtrans_object=='')|(backtrans_object==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans_object = element.text 
                encode_object.append(backtrans_object)
            else:
                encode_object.append(backtrans_object)
            
        except:
            encode_object.append('')
            
    return  encode_sentence, encode_subject, encode_object

In [4]:
def via_trans(sentence, subject_list, object_list, start_lang, end_lang):
    """
    한국어가 아닌 다른언어간 번역
    """
    via_sentence = []
    via_subject = []
    via_object = []
    
    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

    for i in tqdm(range(len(sentence))): 
        if sentence[i] == '' or subject_list[i] == '' or object_list[i] == '':
            via_sentence.append('')
            via_subject.append('')
            via_object.append('')
            continue
        try:
            driver.get('https://papago.naver.com/?sk=' + start_lang +'&tk=' + end_lang + '&st=' + sentence[i])
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.8)
            backtrans_sentence = element.text 

            if (backtrans_sentence=='')|(backtrans_sentence==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans_sentence = element.text 
                via_sentence.append(backtrans_sentence)
            else:
                via_sentence.append(backtrans_sentence)
            
        except:
            via_sentence.append('')
            
        try:
            driver.get('https://papago.naver.com/?sk=' + start_lang +'&tk=' + end_lang + '&st=' + extract_word(subject_list[i]))
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.8)
            backtrans_subject = element.text 

            if (backtrans_subject=='')|(backtrans_subject==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans_subject = element.text 
                via_subject.append(backtrans_subject)
            else:
                via_subject.append(backtrans_subject)
            
        except:
            via_subject.append('')
            
        try:
            driver.get('https://papago.naver.com/?sk=' + start_lang +'&tk=' + end_lang + '&st=' + extract_word(object_list[i]))
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.8)
            backtrans_object = element.text 

            if (backtrans_object=='')|(backtrans_object==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans_object = element.text 
                via_object.append(backtrans_object)
            else:
                via_object.append(backtrans_object)
            
        except:
            via_object.append('')

            
    return  via_sentence, via_subject, via_object

In [5]:
def extract_word(x):
    return x.split(',')[0].split(':')[-1].strip()[1:-1]

def extract_type(x):
    return x.split(',')[-1].split(':')[-1].strip()[1:-1]

In [6]:
def find_index(sentence, word):
    length = len(word)
    start_index = sentence.find(word)
    end_index = start_index + length -1
    return start_index, end_index

In [7]:
# back trnaslation진행할 dataframe 생성
original_df = pd.read_csv('C:/Users/Admin/Desktop/Boostcamp-AI-Tech/dataset/train/train.csv')
train = original_df[original_df['label'] == 'no_relation']

In [8]:
kor_jp_sentence, kor_jp_subject, kor_jp_object = kor_to_trans(train, 'ja&hn=0',0,len(train))

In [11]:
jp_kor_sentence, jp_kor_subject, jp_kor_object = via_trans(kor_jp_sentence, kor_jp_subject, kor_jp_object, 'ja&hn=0', 'ko') 

In [12]:
original_subject = train['subject_entity'].apply(extract_word).tolist()
original_object = train['object_entity'].apply(extract_word).tolist()

In [16]:
id_count = int(32470) # 원본데이터id 0~32469
bs_df = pd.DataFrame()  # bt(back translation) 진행한 문장에 원래 subject,object가 들어있는경우
bs2_df = pd.DataFrame()  # bt진행한 문장에 bt진행한 subject가 들어있고 원래 object가 들어있는경우
bs3_df = pd.DataFrame()  # bt진행한 문장에 원래 subject가 들어있고 bt진행한 object가 들어있는경우
bs4_df = pd.DataFrame()  # bt진행한 문장에 bt진행한 subject,object 들어있는경우
for i in range(len(train)):
    if jp_kor_sentence[i] == '' or jp_kor_subject[i] == '' or jp_kor_object[i] == '':
        continue
        
    if (original_subject[i] in jp_kor_sentence[i]) and (original_object[i] in jp_kor_sentence[i]):
        ss,se = find_index(jp_kor_sentence[i], original_subject[i])
        slabel = extract_type(train['subject_entity'].iloc[i])
        sdict = "{'word': '"+original_subject[i]+"',  \'start_idx\': "+str(ss)+",  \'end_idx\': "+str(se)+", 'type': '"+slabel+"}"
        
        os,oe = find_index(jp_kor_sentence[i], original_object[i])
        olabel = extract_type(train['object_entity'].iloc[i])
        odict = "{'word': '"+original_object[i]+"',  \'start_idx\': "+str(os)+",  \'end_idx\': "+str(oe)+", 'type': '"+olabel+"}"
        new_data = [{
            'id' : id_count,
            'sentence' : jp_kor_sentence[i],
            'subject_entity' : sdict,
            'object_entity' : odict,
            'label' : train['label'].iloc[i],
            'source' : train['source'].iloc[i]
        }]
        new_df = pd.DataFrame(new_data)
        bs_df = pd.concat([bs_df, new_df])
        id_count += 1


    elif jp_kor_subject[i] in jp_kor_sentence[i] and original_object[i] in jp_kor_sentence[i]:
        ss,se = find_index(jp_kor_sentence[i], jp_kor_subject[i])
        slabel = extract_type(train['subject_entity'].iloc[i])
        sdict = "{'word': '"+jp_kor_subject[i]+"',  \'start_idx\': "+str(ss)+",  \'end_idx\': "+str(se)+", 'type': '"+slabel+"}"

        os,oe = find_index(jp_kor_sentence[i], original_object[i])
        olabel = extract_type(train['object_entity'].iloc[i])
        odict = "{'word': '"+original_object[i]+"',  \'start_idx\': "+str(os)+",  \'end_idx\': "+str(oe)+", 'type': '"+olabel+"}"
        

        new_data = [{
            'id' : id_count,
            'sentence' : jp_kor_sentence[i],
            'subject_entity' : sdict,
            'object_entity' : odict,
            'label' : train['label'].iloc[i],
            'source' : train['source'].iloc[i]
        }]
        new_df = pd.DataFrame(new_data)
        bs2_df = pd.concat([bs2_df, new_df])

    elif original_subject[i] in jp_kor_sentence[i] and jp_kor_object[i] in jp_kor_sentence[i]:
        ss,se = find_index(jp_kor_sentence[i], original_subject[i])
        slabel = extract_type(train['subject_entity'].iloc[i])
        sdict = "{'word': '"+original_subject[i]+"',  \'start_idx\': "+str(ss)+",  \'end_idx\': "+str(se)+", 'type': '"+slabel+"}"

        os,oe = find_index(jp_kor_sentence[i], jp_kor_object[i])
        olabel = extract_type(train['object_entity'].iloc[i])
        odict = "{'word': '"+jp_kor_object[i]+"',  \'start_idx\': "+str(os)+",  \'end_idx\': "+str(oe)+", 'type': '"+olabel+"}"

        new_data = [{
            'id' : id_count,
            'sentence' : jp_kor_sentence[i],
            'subject_entity' : sdict,
            'object_entity' : odict,
            'label' : train['label'].iloc[i],
            'source' : train['source'].iloc[i]
        }]
        new_df = pd.DataFrame(new_data)
        bs3_df = pd.concat([bs3_df, new_df])
        
    elif jp_kor_subject[i] in jp_kor_sentence[i] and jp_kor_object[i] in jp_kor_sentence[i]:
        ss,se = find_index(jp_kor_sentence[i], jp_kor_subject[i])
        slabel = extract_type(train['subject_entity'].iloc[i])
        sdict = "{'word': '"+jp_kor_subject[i]+"',  \'start_idx\': "+str(ss)+",  \'end_idx\': "+str(se)+", 'type': '"+slabel+"}"

        os,oe = find_index(jp_kor_sentence[i], jp_kor_object[i])
        olabel = extract_type(train['object_entity'].iloc[i])
        odict = "{'word': '"+jp_kor_object[i]+"',  \'start_idx\': "+str(os)+",  \'end_idx\': "+str(oe)+", 'type': '"+olabel+"}"

        new_data = [{
            'id' : id_count,
            'sentence' : jp_kor_sentence[i],
            'subject_entity' : sdict,
            'object_entity' : odict,
            'label' : train['label'].iloc[i],
            'source' : train['source'].iloc[i]
        }]
        new_df = pd.DataFrame(new_data)
        bs4_df = pd.concat([bs4_df, new_df])

In [17]:
df_list = [bs_df, bs2_df, bs3_df, bs4_df]
result_df = pd.concat(df_list, ignore_index=True)
result_df = result_df.sample(frac=1)  # 행 랜덤으로 셔플
result_df = result_df[result_df['subject_entity'] != result_df['object_entity']]
result_df = result_df.reset_index(drop=True)

In [18]:
df_list = [original_df, merge_df]
merge_df = pd.concat(df_list, ignore_index=True)
merge_df = merge_df.reset_index(drop=True)
merge_df['id'] = list(range(len(merge_df)))

In [89]:
merge_df.to_csv('nore_aug.csv', index=False, encoding="utf-8-sig") 

원래 train.csv 가져와서 합치면 증강된 dataset 완성됨